In [19]:
%matplotlib tk
%load_ext autoreload
%autoreload 2
#
import matplotlib.pyplot as plt

#
from utils import GerbilPCA
import glob
from tqdm import trange
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# # OPTINAL CONVERT SECONDS TO HOURLY ETHOGRAMS
# from utils import generate_hourly_ethogram2

# filename_in = ''
# animal_id = 0

# #
# generate_hourly_ethogram2(filename_in,
#                               animal_id)

In [37]:
fname = '/media/cat/4TBSSD/dan/meta_data/ethogram_seconds_foodhopper_proximity.npy'
rectangle_ethogram = np.load(fname)
print (rectangle_ethogram.shape)

#
plt.figure()
plt.imshow(rectangle_ethogram,
           aspect='auto',
           interpolation='none')
plt.show()

(112, 86400)


In [38]:
# grab the entire ethogram for a single animal;

def get_eth(rectangle_ethogram, id_):

    #
    idxs = np.arange(1,rectangle_ethogram.shape[0],7)+(5-id_)
    idxs = idxs[::-1]

    #
    eth = np.hstack(rectangle_ethogram[idxs])

    #
    return eth

In [84]:
# compute coincidence of time for two animals in the same time bin
clrs = ['red','blue','green','black','magenta','cyan']
hatch_styles = ['/', '\\', '|', '-', '+', 'x', 'o', '*', '.']

#
plt.figure()
ctr2 = 0
for a1 in trange(6):
    eth1 = get_eth(rectangle_ethogram, a1)
    idx3 = np.where(eth1>0)[0]
    ctr=0
    for a2 in range(6):
        if a1==a2:
            ctr+=1
            continue
        #
        eth2 = get_eth(rectangle_ethogram, a2)

        #
        window = 5 # search within a 5 sec window
        s = 0
        for k in idx3:
            if eth1[k]>0:
                # 
                if np.nanmax(eth2[k-window:k+window])>0:
                    s+=1

        # plot bar graph
        plt.bar(a1+ctr2+ctr/6, 
                s, 
                1/6.*.9,
                label=str(a1)+' vs '+str(a2),
                color=clrs[a1],
                hatch=hatch_styles[a2]
                )
        
        ctr+=1
       # break  
    
    #
    ctr2+=1
   # if a1==2:
   #     break

#
plt.legend()
plt.show()


100%|██████████| 6/6 [00:34<00:00,  5.81s/it]


In [54]:
###########################################
########### CORRELATION ANALYSIS ##########
###########################################

# NOT EVERY INSIGHTFUL FOR NOW

# compute a cross correlogram by shifting one ethogram against the other from -30 to +30 seconds
# use pearson correlation 

def get_xcorr(aas, rectangle_ethogram, window):

    #
    a1, a2 = aas

    #
    eth1 = get_eth(rectangle_ethogram, a1)

    #
    eth2 = get_eth(rectangle_ethogram, a2)

    from scipy.stats import pearsonr
    #
    #window = 60
    cc = []
    idx1 = np.where(np.isnan(eth1))[0]
    for k in trange(-window,window,1):

        #
        temp = np.roll(eth2,k).copy()

        # find nans 
        idx2 = np.where(np.isnan(temp))[0]

        # find union of idx1 and idx2
        idx = np.union1d(idx1,idx2)

        # delete idx from both eth1 and temp
        temp2 = np.delete(eth1,idx).copy()
        temp = np.delete(temp,idx).copy()

        #    
        corr, _ = pearsonr(temp2, temp)  
        #print (corr, pva)
        cc.append(corr)

    return cc

#########################################

# 
aas = []
for a1 in range(6):
    for a2 in range(a1+1,6,1):
        aas.append([a1,a2])

#
print ("# of comparisons", len(aas))
#
window = 120

#
import parmap
if True:
    ccs = parmap.map(get_xcorr,
                     aas,
                        rectangle_ethogram,
                        window,
                        pm_processes = 15,
                        pm_pbar = True)
    
#
plt.figure()
t = np.arange(-window,window,1)
ctr=0
for a1 in range(6):
    for a2 in range(a1+1,6,1):
        cc = ccs[ctr]
        #
        plt.plot(t, cc,
                 label = str(a1)+' '+str(a2))
        plt.show()

        ctr+=1

# get y limits from plot 
ymin, ymax = plt.ylim()

# plot vertical line at 0
plt.plot([t[window],t[window]],
            [ymin,ymax],
            '--',
            c='black')

#
plt.legend()
plt.show()




100%|██████████| 15/15 [01:53<00:00,  7.57s/it]


In [40]:
behaviors = [

#################################
####### EXPLORATION TIME ########
#################################
# 'adult_female_exploration',
# 'adult_male_exploration',
# 'pup_female_exploration',
# 'pup_male_exploration',

#################################
### DISTANCE TRAVELLED ##########
#################################
 #'pup_female_distance',
 #'pup_male_distance',
# 'adult_female_distance',
# 'adult_male_distance',

#################################
######## FOOD PROXIMITY #########
#################################
# 'adult_female_food',
# 'adult_male_food',
# 'pup_female_food',
# 'pup_male_food',

#################################
####### WATER PROXIMITY #########
#################################

# 'adult_female_water',
# 'adult_male_water',
# 'pup_female_water',
# 'pup_male_water',


#################################
########## NEST EXITS ###########
#################################

# # SEQUENCES OF EXITS
# 'adult_female_exits',
# 'adult_male_exits',
#  'pup_female_exits',
#  'pup_male_exits',
 
#################################
######### HUDDLING TIME #########
#################################

# # # HUDDLING TIME  - INVERSE OF EXPLORATION TIME <- NO NEED TO SHOW BOTH
# 'adult_female_huddling',
# 'adult_male_huddling',
# 'pup_female_huddling',
# 'pup_male_huddling',


#################################
#### PAIRWISE TIME TOGETHER #####
#################################
# 'femaleadult_femalepup_pairwise',
# 'femaleadult_malepup_pairwise',

# 'maleadult_femalepup_pairwise',
# 'maleadult_malepup_pairwise',

#'malepup_malepup_pairwise',
'femalepup_femalepup_pairwise',
'femalepup_malepup_pairwise',

#################################
########## APPROACHES ###########
#################################

# # ADULT - ADULT approaches
# 'femaleadult_to_maleadult_approaches',
# 'maleadult_to_femaleadult_approaches',

# PUP - PUP approaches
#'femalepup_to_femalepup_approaches',
#'femalepup_to_malepup_approaches',
#'malepup_to_femalepup_approaches',
#'malepup_to_malepup_approaches',

# # ADULT FEMALE - PUP approaches
#'femaleadult_to_femalepup_approaches',
# 'femaleadult_to_malepup_approaches',
# 'malepup_to_femaleadult_approaches',
#'femalepup_to_femaleadult_approaches',

# # ADULT MALE - PUP approaches
#'maleadult_to_femalepup_approaches',
#'maleadult_to_malepup_approaches',
#'malepup_to_maleadult_approaches',
# 'femalepup_to_maleadult_approaches',

]



In [41]:
#
g = GerbilPCA(root_dir, 
              behaviors)

#
g.smooth = False
g.smooth_window = 5
g.smooth_order = 2
g.standardize = True
g.n_dimensions = 2
g.scatter_size = 600

#
g.load_data()

#
g.n_samples_kde = 1000
g.sliding_window_size = 2
g.dist_method = '2sample_ks_test'  # options: t_test, 2sample_ks_test
g.pval_thresh = 0.01
g.show_plots = True
g.smooth_pval = True
g.size=500
g.interpolate_first_value = False
g.get_rapid_dev_plots()

# visualize averages
g.plot_mean_behavior()

# compute areas under curve for pairs
g.get_area_under_curve()
# get exponential notatoin two signifcant figures for g.ks_[1]
text1 = "{:.2e}".format(g.ks_[1])
text2 = "{:.2e}".format(g.ttest[1])
plt.suptitle("ks pval: "+text1 + ", ttest: pval " + text2)


behavior:  femalepup_femalepup_pairwise (10, 14)
behavior:  femalepup_malepup_pairwise (7, 14)


  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<00:00, 273.84it/s]

mean:  1.2702058531746032  std:  0.5071105367325097
mean:  0.7802437641723354  std:  0.45444027526429814


auc_f mean:  16.834131944444444  ,std:  2.8278842364350507
auc_m mean:  10.17986111111111  ,std:  2.2649103773231287


Text(0.5, 0.98, 'ks pval: 8.23e-04, ttest: pval 2.09e-04')

In [30]:
# compute areas under the curve for each animal
g.get_area_under_curve()

temp_stack:  (6, 14)
temp_starts:  (2, 2)
[[0 3]
 [3 6]]
ks test:  KstestResult(statistic=1.0, pvalue=0.09999999999999999)
ttest:  Ttest_indResult(statistic=2.0072925072275902, pvalue=0.11515405792257753)


In [ ]:
#
g.show_convex_hull = True
g.remove_outlier_point= False
g.show_cohorts = False
g.run_pca()

In [113]:
#
g.find_overlaps()



In [39]:
fname = '/home/cat/code/gerbil/meta_data/fig3/malepup_malepup_pairwise.npy'
d = np.load(fname)

print (d.shape)
d = np.mean(d, axis=2)
print (d.shape)
np.save(fname, d)

(1, 14, 24)
(1, 14)


In [31]:
#
#g.run_umap()

In [10]:

d = np.load('/home/cat/code/gerbil/meta_data/fig3/malepup_malepup_pairwise.npy')

print (d.shape)

#np.save('/home/cat/code/gerbil/meta_data/fig3/malepup_malepup_pairwise.npy', d)


(1, 14)


In [26]:
g.n_dimensions = 2
g.fit_gmm_pca()



(48, 14)
(48, 2)


In [27]:
#


(48, 2)


In [151]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

# Step 1: Generate or load your 3D data (replace this with your actual data)
np.random.seed(0)
data = np.random.rand(50, 3)  # 50 points in 3D

# Step 2: Compute the convex hull
hull = ConvexHull(data)
vertices = data[hull.vertices]  # Vertices of the convex hull

# get the ids of the vertices
# make a larger vertices array and fill in the missing vertices with zeros
vertices = np.zeros((data.shape[0],3))
vertices[hull.vertices] = data[hull.vertices]

print (vertices.shape)

#

# Step 3: Extract the valid edges and vertices from the convex hull
edges = []
for simplex in hull.simplices:
    n_bad = 0
    for v in simplex:
        #print (v)
        if v < 0 or v >= len(data):
            n_bad += 1

    if n_bad == 0:
        edges.append(simplex)

# Step 4: Create the 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Step 5: Create a Poly3DCollection for the 3D polygon
poly3d = [[vertices[edge[0]], vertices[edge[1]], vertices[edge[2]]] for edge in edges]

ax.add_collection3d(Poly3DCollection(poly3d, 
                                     facecolors='black', 
                                     linewidths=1, 
                                     edgecolors='black', 
                                     alpha=0.25))

# Step 6: Set axis labels and display the plot
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')

plt.show()


(50, 3)


In [254]:
import vedo

# Define the vertices of the first polyhedron
vertices1 = [
    [0, 0, 0],
    [1, 0, 0],
    [1, 1, 0],
    [0, 1, 0],
    [0, 0, 1],
    [1, 0, 1],
    [1, 1, 1],
    [0, 1, 1]
]

# Define the faces for the first polyhedron
faces1 = [
    [0, 1, 2, 3],  # Bottom face
    [4, 5, 6, 7],  # Top face
    [0, 1, 5, 4],  # Side 1
    [1, 2, 6, 5],  # Side 2
    [2, 3, 7, 6],  # Side 3
    [3, 0, 4, 7]   # Side 4
]

# Define the vertices of the second polyhedron
vertices2 = [
    [0.5, 0.5, 0],
    [1.5, 0.5, 0],
    [1.5, 1.5, 0],
    [0.5, 1.5, 0],
    [0.5, 0.5, 1],
    [1.5, 0.5, 1],
    [1.5, 1.5, 1],
    [0.5, 1.5, 1]
]

# Define the faces for the second polyhedron
faces2 = [
    [0, 1, 2, 3],  # Bottom face
    [4, 5, 6, 7],  # Top face
    [0, 1, 5, 4],  # Side 1
    [1, 2, 6, 5],  # Side 2
    [2, 3, 7, 6],  # Side 3
    [3, 0, 4, 7]   # Side 4
]

# Create the two polyhedron objects
polyhedron1 = vedo.shapes.Poly(vertices1, faces1)
polyhedron2 = vedo.shapes.Poly(vertices2, faces2)

# Compute the intersection between the two polyhedrons
intersection = polyhedron1.intersectWith(polyhedron2)

# Visualize the intersection
show(intersection)


AttributeError: module 'vedo.shapes' has no attribute 'Poly'